In [18]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,f1_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [19]:
train=pd.read_csv("train_mod.csv")
test=pd.read_csv("test_mod.csv")

#label encoding the features for the modelling process
le=LabelEncoder()
le.fit(train["color_type"])
# train["color_type"]=le.fit_transform(train["color_type"])
train["color_type"]=le.transform(train["color_type"])
test["color_type"]=le.transform(test["color_type"])

In [20]:
train_set,val_set=train_test_split(train,test_size=.25,random_state=42)
train_set.shape,val_set.shape

#-------------------------------------------------------------------------------
#train_set
train_data=train_set.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed']
                          ,axis=1)
train_res_breed=train_set.breed_category
train_res_pet=train_set.pet_category

#-------------------------------------------------------------------------------
#val_set
val_data=val_set.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed'],
                     axis=1)
val_res_breed=val_set.breed_category
val_res_pet=val_set.pet_category
#-------------------------------------------------------------------------------
train_full=train.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed']
                          ,axis=1)
train_full_breed=train.breed_category
train_full_pet=train.pet_category
test_full=test.drop(['pet_id', 'issue_date', 'listing_date','days_stayed']
                          ,axis=1)

#-------------------------------------------------------------------------------

train_data.shape,train_res_breed.shape,val_data.shape,val_res_breed.shape,train_full.shape,test_full.shape

((14125, 9), (14125,), (4709, 9), (4709,), (18834, 9), (8072, 9))

In [21]:
lgbc=LGBMClassifier()

In [22]:
models=[lgbc]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ensemble_for_train_val(X_train,y_train,X_test, y_test):
    for model in models:
        print(model.__class__.__name__)
        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)
        y_test_pred_new=model.predict_proba(X_test)
        y_test_pred_new=y_test_pred_new[:,1]
        train_score=model.score(X_train,y_train)
        test_score=model.score(X_test,y_test)
        p_score=metrics.precision_score(y_test,y_test_pred,average='micro')
        r_score=metrics.recall_score(y_test,y_test_pred,average='micro')
        
        sctr.append(train_score)
        scte.append(test_score)
        ps.append(p_score)
        rs.append(r_score)
    return sctr,scte,ps,rs




In [23]:

ensemble_for_train_val(train_data,train_res_breed, val_data, val_res_breed)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'

ensemble=pd.DataFrame({'names':["lgbc"],
                       'training':sctr,'testing':scte,'precision':ps,'recall':rs,})

ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
ensemble

LGBMClassifier


,names,training,testing,precision,recall
0,lgbc,0.976212,0.901253,0.901253,0.901253


In [ ]:

# ensemble_for_train_val(train_data,train_res_pet, val_data, val_res_pet)
# # 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'

# ensemble=pd.DataFrame({'names':["lgbc"],
#                        'training':sctr,'testing':scte,'precision':ps,'recall':rs,})

# ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
# ensemble

param_dict={
    'boosting_type' :["dart"],
    'is_unbalance':[True],
    'n_estimators':[400,500,600],
    'num_leaves':[10,20], 
    'max_depth':[8,10,12],
#     'feature_fraction':[0.5,.75],  
#     'bagging_fraction':[0.8,.5], 
#     'bagging_freq':[15,20], 
    'learning_rate':[0.01,.1]
}

gridSearchCV = GridSearchCV(estimator = lgbc, 
    param_grid = param_dict, 
    scoring='accuracy',
    n_jobs=-1,
    iid=False, 
    verbose=1,
    cv=3)
gridSearchCV.fit(train_full,train_full_pet)
gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
gridSearchCV.best_params_, gridSearchCV.best_score_